# 已有该学生历史成绩，规划学习路径

In [ ]:
# MATCH (K1:Knowledge {name:'平行四边形计算公式'})-[relN:NEXT]- (K2:Knowledge) return K1,K2,relN
# 注意：数据因为是模拟生成的不一定精准， 得到平行四边形计算前置的知识点包括：整数加、减、乘、除及常用面积单位，并得到影响权重系数
# 根据学生历史成绩预测 本知识点可能的掌握情况， 并根据学生在该知识点想达到的目标，得到其它的前置知识点需要的最低掌握情况
'''
处理流程：
1、训练贝叶斯模型（根据历史所有成绩）
2、根据该学生成绩，预测知识点的掌握情况。（？是否可以根据权重系数来判断，？？？？？？）
'''

# 无该学生历史成绩，预测学生前置知识点掌握情况

In [ ]:
# 无学生历史成绩， 根据学生在该知识点的掌握情况（通过测试得到），得到其它前置知识点掌握情况
